In [1]:
cd ../..

/home/nazif/thesis/mirscribe-vcf


/home/nazif/.cache/pypoetry/virtualenvs/mirscribe-vcf-8VJm4tPg-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from sqlalchemy import create_engine

import pandas as pd

# db engine
engine = create_engine('sqlite:///data/db/mirscribe.db')



# mirnas

In [3]:
mirna_df = pd.read_csv('data/mirna/mirna.csv')
mirna_df["seed"] = mirna_df.sequence.str[1:8]

mirna_df.head()

,mirna_name,mirna_accession,sequence,mirna_family,conservation,seed
0,hsa-let-7a-5p,MIMAT0000062,AACTATACAACCTACTACCTCA,let-7-5p/98-5p,2.0,ACTATAC
1,hsa-let-7b-5p,MIMAT0000063,AACCACACAACCTACTACCTCA,let-7-5p/98-5p,2.0,ACCACAC
2,hsa-let-7c-5p,MIMAT0000064,AACCATACAACCTACTACCTCA,let-7-5p/98-5p,2.0,ACCATAC
3,hsa-let-7d-5p,MIMAT0000065,AACTATGCAACCTACTACCTCT,let-7-5p/98-5p,2.0,ACTATGC
4,hsa-let-7e-5p,MIMAT0000066,AACTATACAACCTCCTACCTCA,let-7-5p/98-5p,2.0,ACTATAC


In [4]:
mirna_df.isna().sum()

mirna_name         0
mirna_accession    0
sequence           0
mirna_family       0
conservation       0
seed               0
dtype: int64

# cancer mirna consensus

In [5]:
cmc = pd.read_csv("data/mirna_cmc/cmc_mirnas.csv")

rename_dict = {
    'miRNA precursor/locus ID (miRBase)': 'miRNA_ID_miRBase',
    'miRNA ID (miRBase)': 'mirna_name_cmc',
    'miRNA precursor/locus ID (MirGeneDB)': 'miRNA_ID_MirGeneDB',
    'oncogene (O)/tumor-suppressor (TS)': 'oncogene_tumor_suppressor'
}

cmc = (
    cmc.rename(columns=rename_dict)
    .assign(mirna_name_cmc=lambda x: 'hsa-' + x['mirna_name_cmc'].astype(str))
    .assign(is_mirna_oncogene=lambda x: x['oncogene_tumor_suppressor'] == 'O',
            is_mirna_tumor_suppressor=lambda x: x['oncogene_tumor_suppressor'] == 'TS')
    .drop(columns=['oncogene_tumor_suppressor'])
    [['mirna_name_cmc', 'is_mirna_oncogene', 'is_mirna_tumor_suppressor']]
    .drop_duplicates()
)

cmc.head()


,mirna_name_cmc,is_mirna_oncogene,is_mirna_tumor_suppressor
0,hsa-miR-100,False,True
1,hsa-miR-101,False,True
3,hsa-miR-103a,True,False
5,hsa-miR-106a,False,False
6,hsa-miR-106b,True,False


In [6]:
def modify_mirna_name(row):
    return row[:-3] if row.endswith("p") else row

mirna_df['mirna_name_cmc'] = mirna_df['mirna_name'].apply(modify_mirna_name)

In [7]:
cmc.mirna_name_cmc.isin(mirna_df.mirna_name_cmc).value_counts()


mirna_name_cmc
True    134
Name: count, dtype: int64

In [8]:
mirna_df = mirna_df.merge(cmc, on='mirna_name_cmc', how='left')
mirna_df.fillna({'is_mirna_oncogene': False, 'is_mirna_tumor_suppressor': False}, inplace=True)

/tmp/ipykernel_79580/534122014.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mirna_df.fillna({'is_mirna_oncogene': False, 'is_mirna_tumor_suppressor': False}, inplace=True)


In [9]:
mirna_df

,mirna_name,mirna_accession,sequence,mirna_family,conservation,seed,mirna_name_cmc,is_mirna_oncogene,is_mirna_tumor_suppressor
0,hsa-let-7a-5p,MIMAT0000062,AACTATACAACCTACTACCTCA,let-7-5p/98-5p,2.0,ACTATAC,hsa-let-7a,False,False
1,hsa-let-7b-5p,MIMAT0000063,AACCACACAACCTACTACCTCA,let-7-5p/98-5p,2.0,ACCACAC,hsa-let-7b,False,True
2,hsa-let-7c-5p,MIMAT0000064,AACCATACAACCTACTACCTCA,let-7-5p/98-5p,2.0,ACCATAC,hsa-let-7c,False,True
3,hsa-let-7d-5p,MIMAT0000065,AACTATGCAACCTACTACCTCT,let-7-5p/98-5p,2.0,ACTATGC,hsa-let-7d,False,False
4,hsa-let-7e-5p,MIMAT0000066,AACTATACAACCTCCTACCTCA,let-7-5p/98-5p,2.0,ACTATAC,hsa-let-7e,False,True
...,...,...,...,...,...,...,...,...,...
2651,hsa-miR-12131,MIMAT0049025,TGGGAGTACACCTCTCCAAA,miR-12131_pred,-1.0,GGGAGTA,hsa-miR-12131,False,False
2652,hsa-miR-12132,MIMAT0049026,CATCATAATTCTCACAGTAATA,miR-12132_pred,2.0,ATCATAA,hsa-miR-12132,False,False
2653,hsa-miR-12133,MIMAT0049027,TGTACTTTTTAATGGTGCCAAG,miR-12133_pred,2.0,GTACTTT,hsa-miR-12133,False,False
2654,hsa-miR-12135,MIMAT0049031,TTTACAAACAAACCTTTA,miR-12135_pred,-1.0,TTACAAA,hsa-miR-12135,False,False


# SAVE TO DB

In [12]:
mirna_df.to_sql('mirnas', engine, if_exists='replace', index=False)
 

2656